In [32]:
##IMPORTS
from __future__ import division
import rhinoinside
rhinoinside.load()

import clr
clr.AddReference("Grasshopper")
from Grasshopper import DataTree
from Grasshopper.Kernel.Data import GH_Path as Path

import System
import Rhino.Geometry as rg
from Rhino.Geometry import Point3d, Plane, Brep, Vector3d
from Rhino.Geometry import Surface as RhinoSurface

from Rhino.Geometry import Transform
from Rhino.FileIO import File3dm

import ladybug.epw as epw

# from ladybug_geometry.geometry2d.pointvector import Vector2D

from lbt_recipes.version import check_radiance_date

#IMPORTS FOR INCIDENT RADIATION
# from ladybug.viewsphere import view_sphere
# from ladybug.graphic import GraphicContainer
# from ladybug.legend import LegendParameters
# from ladybug.color import Colorset

# from ladybug_rhino.config import conversion_to_meters
# from ladybug_rhino.togeometry import to_joined_gridded_mesh3d, to_vector2d
from ladybug_rhino.fromgeometry import from_mesh3d, from_point3d, from_vector3d
# from ladybug_rhino.intersect import intersect_mesh_rays, join_geometry_to_mesh

from datetime import datetime, timedelta
import array
import math

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from honeybee.model import Model
from honeybee.face import Face
from honeybee_radiance.sensorgrid import SensorGrid
from honeybee.typing import clean_and_id_string

from ladybug_geometry.geometry3d.face import Face3D
# from ladybug_geometry.geometry3d.pointvector import Point3D
from ladybug_geometry.geometry3d.plane import Plane
from ladybug_geometry.geometry3d.face import Face3D
from ladybug_geometry.geometry3d.mesh import Mesh3D
from ladybug_rhino.togeometry import  to_mesh3d, to_face3d, to_vector3d
from ladybug_rhino.fromgeometry import from_point3d

from lbt_recipes.recipe import Recipe
from lbt_recipes.settings import RecipeSettings
from ladybug_rhino.togeometry import to_face3d
from ladybug.wea import Wea


import os
import shutil
# check the installed Radiance date and get the path to the gemdaymtx executable
check_radiance_date()

In [33]:
#FONCTIONS GH & LB 
def objectify_output(object_name, output_data):
    """Wrap data into a single custom Python object that can later be de-serialized.

    This is meant to address the same issue as the wrap_output method but it does
    so by simply hiding the individual items from the Grasshopper UI within a custom
    parent object that other components can accept as input and de-objectify to
    get access to the data. This strategy is also useful for the case of standard
    object types like integers where the large number of data points slows down
    the Grasshopper UI when they are output.

    Args:
        object_name: Text for the name of the custom object that will wrap the data.
            This is how the object will display in the Grasshopper UI.
        output_data: A list of data to be stored under the data property of
            the output object.
    """
    class Objectifier(object):
        """Generic class for objectifying data."""

        def __init__(self, name, data):
            self.name = name
            self.data = data

        def ToString(self):
            return '{} ({} items)'.format(self.name, len(self.data))

    return Objectifier(object_name, output_data)

def de_objectify_output(objectified_data):
    """Extract the data from an object that was output from the objectify_output method.

    Args:
        objectified_data: An object that has been output from the objectify_output
            method for which data will be returned.
    """
    return objectified_data.data

def text_objects(text, plane, height, font='Arial',
                 horizontal_alignment=0, vertical_alignment=5):
    """Generate a Bake-able Grasshopper text object from a text string and ladybug Plane.

    Args:
        text: A text string to be converted to a a Grasshopper text object.
        plane: A Ladybug Plane object to locate and orient the text in the Rhino scene.
        height: A number for the height of the text in the Rhino scene.
        font: An optional text string for the font in which to draw the text.
        horizontal_alignment: An optional integer to specify the horizontal alignment
             of the text. Choose from: (0 = Left, 1 = Center, 2 = Right)
        vertical_alignment: An optional integer to specify the vertical alignment
             of the text. Choose from: (0 = Top, 1 = MiddleOfTop, 2 = BottomOfTop,
             3 = Middle, 4 = MiddleOfBottom, 5 = Bottom, 6 = BottomOfBoundingBox)
    """
    txt = Rhino.Display.Text3d(text, from_plane(plane), height)
    txt.FontFace = font
    txt.HorizontalAlignment = AlignmentTypes.horizontal(horizontal_alignment)
    txt.VerticalAlignment = AlignmentTypes.vertical(vertical_alignment)
    return TextGoo(txt)

def longest_list(values, index):
    """Get a value from a list while applying Grasshopper's longest-list logic.

    Args:
        values: An array of values from which a value will be pulled following
            longest list logic.
        index: Integer for the index of the item in the list to return. If this
            index is greater than the length of the values, the last item of the
            list will be returned.
    """
    try:
        return values[index]
    except IndexError:
        return values[-1]

def to_gridded_mesh3d_perso(brep, grid_size, offset_distance=0):
    """Create a gridded Ladybug Mesh3D from a Rhino Brep.

    This is useful since Rhino's grid meshing is often more beautiful than what
    ladybug_geometry can produce. However, the ladybug_geometry Face3D.get_mesh_grid
    method provides a workable alternative to this if it is needed.

    Args:
        brep: A Rhino Brep that will be converted into a gridded Ladybug Mesh3D.
        grid_size: A number for the grid size dimension with which to make the mesh.
        offset_distance: A number for the distance at which to offset the mesh from
            the underlying brep. The default is 0.
    """
    if not isinstance(brep, rg.Brep):  # it's likely an extrusion object
        brep = brep.ToBrep()  # extrusion objects must be cast to Brep in Rhino 8
    meshing_param = rg.MeshingParameters.Default
    meshing_param.MaximumEdgeLength = grid_size
    meshing_param.MinimumEdgeLength = grid_size
    meshing_param.GridAspectRatio = 1
    mesh_grids = rg.Mesh.CreateFromBrep(brep, meshing_param)
    if len(mesh_grids) == 1:  # only one mesh was generated
        mesh_grid = mesh_grids[0]
    else:  # join the meshes into one
        mesh_grid = rg.Mesh()
        for m_grid in mesh_grids:
            mesh_grid.Append(m_grid)
    if offset_distance != 0:
        temp_mesh = rg.Mesh()
        mesh_grid.Normals.UnitizeNormals()
        for pt, vec in zip(mesh_grid.Vertices, mesh_grid.Normals):
            # Startmodif
            vec3d = rg.Vector3d(vec) * offset_distance
            pt3d = rg.Point3d(pt)
            temp_mesh.Vertices.Add(pt3d + vec3d)
            # Endmodif
        for face in mesh_grid.Faces:
            temp_mesh.Faces.AddFace(face)
        mesh_grid = temp_mesh
    return to_mesh3d(mesh_grid)

def recipe_result(result):
    """Process a recipe result and handle the case that it's a list of list.

    Args:
        result: A recipe result to be processed.
    """
    if isinstance(result, (list, tuple)):
        return list_to_data_tree(result, s_type = str)
    return result

def list_to_data_tree(input, root_count=0, s_type = object):
    """Transform nested of lists or tuples to a Grasshopper DataTree.

    Args:
        input: A nested list of lists to be converted into a data tree.
        root_count: An integer for the starting path of the data tree.
        s_type: An optional data type (eg. float, int, str) that defines all of the
            data in the data tree. The default (object) works will all data types
            but the conversion to data trees can be more efficient if a more
            specific type is specified.
    """

    def proc(input, tree, track):
        for i, item in enumerate(input):
            if isinstance(item, (list, tuple, array.array)):  # ignore iterables like str
                track.append(i)
                proc(item, tree, track)
                track.pop()
            else:
                tree.Add(item, Path(*track))

    if input is not None:
        if not isinstance(s_type, type):
            raise TypeError("s_type must be a valid type")
        t = DataTree[s_type]()
        proc(input, t, [root_count])
        return t

In [34]:
#FONCTIONS GEOMETRIE
def create_surface(x, y, width, length):
    """
    Create a simple NURBS Rhino Surface, from one of its four angles and its dimensions (width, length)

    Args :
    x (float) : X-coordinate of origin angle
    y (float) : Y-coordinate of origin angle
    width (float) : width of the surface
    length (float) : length of the surface

    Return :
    surface (Rhino.Geometry.NurbsSurface) : Surface created
    """
    pt1 = rg.Point3d(x, y, 0)
    pt2 = rg.Point3d(x, y + width, 0)
    pt3 = rg.Point3d(x + length, y + width, 0)
    pt4 = rg.Point3d(x + length, y, 0)
    
    # Create a surface from the previous point list, and return it
    surface = rg.NurbsSurface.CreateFromCorners(pt1, pt2, pt3, pt4)

    return surface
    
def create_panel_grid(grid_size, nb_rangs, nb_pvp_rangs, width, length, height, column_spacing, row_spacing, angle_orientation, angle_variable, semi_transp = False, bande = None, void_space = None):
    """
    According to the parameters, creates a grid of surfaces, later converted into Breps, representing the solar panels.

    Args :
    grid_size (float) : total size of the panel grid ofr simulation (can and will differ from the actual solar panel field)
    nb_rangs (int) : Number of PVP per rows
    nb_pvp_rangs (int) : Number of rows
    width (float) : Width of a PVP
    length (float) : Length of a PVP
    column_spacing (float) : Distance between two PVP rows (forced value (= ENTRAXE))
    row_spacing (float) : Distance between two PVP inside a row (can be equals to 0, forced value (=grid_size/nb_rangs))
    angle_orientation (int) : Difference between the field axis (PVP supports' axis) and the North-South axis
    angle_variable (int) : tilt of the PVPs
    semi_transp (bool) : False = classic PV Panel, one surface
                         True = Semi-transparent PV panel, composed by 4 strips of PV cells and 3 strips of space inbetween the cells.
    bande (float) : width of the PV cells strip, in case of semi transp panel
    void_space (float) : width of the void between PV cells strip, in case of semi transp panel 
    
    Return :
    final_brep (Rhino.Geometry.Brep) : Brep containing all the PVPs

    """
    final_brep = rg.Brep()

    for i in range(nb_rangs):
        for j in range(nb_pvp_rangs):
            # (x;y) are the coordinate of the current panel created
            x = (j * column_spacing) - grid_size / 2
            y = (i * row_spacing) - grid_size / 2
            surface = []

            if semi_transp == True:
                for h in range(0,4) :
                    strip_x = x + h * (bande + void_space)
                    surface.append(create_surface(strip_x, y, length, bande))
            else:    
                surface.append(create_surface(x, y, length, width))
                
            # Rotation around the X axis for each Brep individually
            center_of_surface = Point3d(x + width / 2, y + length / 2, 0)
            rotation_x = Transform.Rotation(angle_variable * (System.Math.PI / 180), Vector3d.YAxis, center_of_surface)
            for surf in surface:
                surf.Transform(rotation_x)
                # Adding it to the final returned brep
                final_brep.Append(surf.ToBrep())       
 
    # Global rotation around the Z axis at the center of the grid (origin)
    rotation_z = Transform.Rotation(-angle_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
    final_brep.Transform(rotation_z)

    # Global translation on the Z axis, equivalent to the panel rotation axis height
    translation_z = Transform.Translation(0,0,height)
    final_brep.Transform(translation_z)

    # Inversion of the orientation of the normals for proper operation of the Incident Radiation function
    final_brep.Flip()

    return final_brep

def create_sensor_grid_2(land_orientation, x, y, culture = False, entraxe = None, rampant = None, nb_lignes = None, grid_size = None):
    """
    Creates the surface from which the Incident Radiation will measure the ground irradiance.

    Args :
    land_orientation (int) : Difference between the field axis (PVP supports' axis) and the North-South axis.
    culture (bool): If True, the measure surface will correspond to a rectangle that will fit under an arboriculture solar panel (used in vine for now)
                    If False, it will correspond to a square, that we could use in field crops.
    x (float) : X-coordinate of the area of measures.
    y (float) : Y-coordinate of the area of measures.
    entraxe (float) : distance between 2 rows of panels supports. None if culture is not arboriculture.
    rampant (float) : width of a pvp panel. None if culture is not arboriculture.
    nb_lignes (int) : number of pvp panels rows (= int(GRID_SIZE / ENTRAXE), calculated in settings)
    grid_size (float) : 
    
    Return:
    sensor_brep_list (list(Rhino.Geometry.Brep)) : Surface created from create_surface function, then converted to a brep, then rotated and translated
                                                    if in the case of arboriculture.

    """
    brep_gen = rg.Brep()
    sensor_brep_list = []
    _TAILLE_SENSOR = 5 #Sensor side length = 5 mètres

    if culture == True:
        # Creating the coordinates of the first point of a measure grid that fits under an arboriculture PVP 
        # (irradiance is not interesting in between vine rows)
        if type(x) == type(None) :
            x_arbo = math.floor(nb_lignes/2)*entraxe + rampant/2 - 0.5 - grid_size/2
            y_arbo = -2.5
        else : x_arbo, y_arbo = x, y

        # Create a surface at the calculated more or less center of the grid UNDER a panel, length = 5 and width = 1 m
        s = create_surface(x_arbo,y_arbo,_TAILLE_SENSOR,1) 
        brep_gen.Append(s.ToBrep())
        
        # Rotating the measure area to align it with the panels, using the same method and geometry references (ZAxis and Origin)
        rotation_z = rg.Transform.Rotation(-land_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
        brep_gen.Transform(rotation_z)

    if culture == False:
        # Create a square with TAILLE_SENSOR is the size of a side, center of the square is the origin of the Rhino Environment
        surface_brep = create_surface(-_TAILLE_SENSOR / 2, -_TAILLE_SENSOR / 2 , _TAILLE_SENSOR, _TAILLE_SENSOR)
        brep_gen.Append(surface_brep.ToBrep())
    
    brep_gen.Flip()
    sensor_brep_list.append(brep_gen)

    return sensor_brep_list

def export_brep_to_file(brep, filename, new_file = True):
    """
    Creates a 3dm file to visualize the objects, mainly Breps, created wiht python scripts.

    Args:
    brep (Rhino.Geometry.Brep): Brep to visualize
    filename (str): Name of the output file.

    Return:
    Nothing, the file is created

    """
    if new_file == False:
        file = File3dm.Read(filename)
        if brep:
            file.Objects.AddBrep(brep)
            file.Write(filename,7)  # Rewrite the file with the new geometry
            print("La nouvelle géométrie a été ajoutée au fichier existant.")
        else:
            print("Erreur lors de la création de la surface.")

    else :
        file = File3dm()
        file.Settings.ModelUnitSystem = Rhino.UnitSystem.Meters
        file.Objects.AddBrep(brep)
        file.Write(filename, 7)  # 7 corresponding to Rhino 7

In [35]:
#FONCTIONS HONEYBEE GRID & MODEL
def combine_grid_model(_model, grids_, views_ = []):
    """
    Add radiance Sensor Grids and/or Views to a Honeybee Model.
    _
    This assignment is necessary for any Radiance study, though whether a grid or a
    view is required for a particular type of study is depenednet upon the recipe
    used.
    _
    Multiple copies of this component can be used in series and each will add the
    grids or views to any that already exist.

    -

        Args:
            _model: A Honeybee Model to which the input grids_ and views_ will be assigned.
            grids_: A list of Honeybee-Radiance SensorGrids, which will be assigned to
                the input _model.
            views_: A list of Honeybee-Radiance Views, which will be assigned to the
                input _model.

        Returns:
            model: The input Honeybee Model with the grids_ and views_ assigned to it.
"""
    assert isinstance(_model, Model), \
        'Expected Honeybee Model. Got {}.'.format(type(_model))
    model = _model.duplicate()  # duplicate to avoid editing the input
    if len(grids_) != 0: # add grids_ if non null
        model.properties.radiance.add_sensor_grids(grids_)
    if len(views_) != 0: # add views_ if non null
        model.properties.radiance.add_views(views_)

    return model

def convert_to_model(faces_, rooms_ = None, shades_ = None, apertures_ = None, doors_ = None, _name_ = "Grille"):
    """
    Create a Honeybee Model, which can be sent for simulation.
    -

        Args:
            rooms_: A list of honeybee Rooms to be added to the Model. Note that at
                least one Room is necessary to make a simulate-able energy model.
            faces_: A list of honeybee Faces to be added to the Model. Note that
                faces without a parent Room are not allowed for energy models.
            shades_: A list of honeybee Shades to be added to the Model.
            apertures_: A list of honeybee Apertures to be added to the Model. Note
                that apertures without a parent Face are not allowed for energy models.
            doors_: A list of honeybee Doors to be added to the Model. Note
                that doors without a parent Face are not allowed for energy models.
            _name_: Text to be used for the Model name and to be incorporated into a unique
                model identifier. If no name is provided, it will be "unnamed" and
                a unique model identifier will be auto-generated.

        Returns:
            report: Reports, errors, warnings, etc.
            model: A Honeybee Model object possessing all of the input geometry
                objects.
    """
    try:  # import the ladybug_rhino dependencies
        from ladybug_rhino.config import units_system, tolerance, angle_tolerance
        from honeybee.typing import clean_string, clean_and_id_string
    except ImportError as e:
        raise ImportError('\nFailed to import ladybug_rhino:\n\t{}'.format(e))
    
    # set a default name and get the Rhino Model units
    name = clean_string(_name_) if _name_ is not None else clean_and_id_string('unnamed')
    units = units_system()

    # create the model
    model = Model(name, rooms_, faces_, shades_, apertures_, doors_,
                units=units, tolerance=tolerance, angle_tolerance=angle_tolerance)

    return model

def brep_to_face(_geo,_name_ = [], _type_ = None, _bc_ = None):
    """
    Create Honeybee Face
    -

        Args:
            _geo: Rhino Brep or Mesh geometry.
            _name_: Text to set the name for the Face and to be incorporated into
                unique Face identifier. If the name is not provided, a random name
                will be assigned.
            _type_: Text for the face type. The face type will be used to set the
                material and construction for the surface if they are not assigned
                through the inputs below. The default is automatically set based
                on the normal direction of the Face (up being RoofCeiling, down
                being Floor and vertically-oriented being Wall).
                Choose from the following:
                    - Wall
                    - RoofCeiling
                    - Floor
                    - AirBoundary
            _bc_: Text for the boundary condition of the face. The boundary condition
                is also used to assign default materials and constructions as well as
                the nature of heat excahnge across the face in energy simulation.
                Default is Outdoors unless all vertices of the geometry lie below
                the below the XY plane, in which case it will be set to Ground.
                Choose from the following:
                    - Outdoors
                    - Ground
                    - Adiabatic
            ep_constr_: Optional text for the Face's energy construction to be looked
                up in the construction library. This can also be a custom OpaqueConstruction
                object. If no energy construction is input here, the face type and
                boundary condition will be used to assign a default.
            rad_mod_: Optional text for the Face's radiance modifier to be looked
                up in the modifier library. This can also be a custom modifier object.
                If no radiance modifier is input here, the face type and boundary
                condition will be used to assign a default.

        Returns:
            report: Reports, errors, warnings, etc.
            face: Honeybee faces. These can be used directly in radiance simulations
                or can be added to a Honeybee room for energy simulation.
    """
    faces = []  # list of faces that will be returned
    for j, geo in enumerate(_geo):
        if len(_name_) == 0:  # make a default Face name
            name = display_name = clean_and_id_string('Face')
        else:
            display_name = '{}_{}'.format(longest_list(_name_, j), j + 1) \
                if len(_name_) != len(_geo) else longest_list(_name_, j)
            name = clean_and_id_string(display_name)
        typ = None
        bc = None
        
        lb_faces = to_face3d(geo)
        for i, lb_face in enumerate(lb_faces):
            face_name = '{}_{}'.format(name, i) if len(lb_faces) > 1 else name
            hb_face = Face(face_name, lb_face, typ, bc)
            hb_face.display_name = display_name

            faces.append(hb_face)  # collect the final Faces
        
    return faces

def brep_to_pts_mesh(_geometry, _grid_size, _offset_dist_ = 1, quad_only_ = False):
    """
    Genrate a mesh with corresponding test points from a Rhino Brep (or Mesh).
    _
    The resulting mesh will be in a format that the "LB Spatial Heatmap" component
    will accept.
    -

        Args:
            _geometry: Brep or Mesh from which to generate the points and grid.
            _grid_size: Number for the size of the test grid.
            _offset_dist_: Number for the distance to move points from the surfaces
                of the input _geometry.  Typically, this should be a small positive
                number to ensure points are not blocked by the mesh. (Default: 0).
            quad_only_: Boolean to note whether meshing should be done using Rhino's
                defaults (False), which fills the entire _geometry to the edges
                with both quad and tringulated faces, or a mesh with only quad
                faces should be generated.
                _
                FOR ADVANCED USERS: This input can also be a vector object that will
                be used to set the orientation of the quad-only grid. Note that,
                if a vector is input here that is not aligned with the plane of
                the input _geometry, an error will be raised.

        Returns:
            points: Test points at the center of each mesh face.
            vectors: Vectors for the normal direction at each of the points.
            face_areas: Area of each mesh face.
            mesh: Analysis mesh that can be passed to the "LB Spatial Heatmap" component.
    """
    # check the input and generate the mesh.
    _offset_dist_ = _offset_dist_ or 0
    if quad_only_:  # use Ladybug's built-in meshing methods
        lb_faces = to_face3d(_geometry)
        try:
            x_axis = to_vector3d(quad_only_)
            lb_faces = [Face3D(f.boundary, Plane(f.normal, f[0], x_axis), f.holes)
                        for f in lb_faces]
        except AttributeError:
            pass  # no plane connected; juse use default orientation
        lb_meshes = []
        for geo in lb_faces:
            try:
                lb_meshes.append(geo.mesh_grid(_grid_size, offset=_offset_dist_))
            except AssertionError:  # tiny geometry not compatible with quad faces
                continue
        if len(lb_meshes) == 0:
            lb_mesh = None
        elif len(lb_meshes) == 1:
            lb_mesh = lb_meshes[0]
        elif len(lb_meshes) > 1:
            lb_mesh = Mesh3D.join_meshes(lb_meshes)
    else:  # use Rhino's default meshing
        lb_mesh = to_gridded_mesh3d_perso(_geometry, _grid_size, _offset_dist_)
    
    # generate the test points, vectors, and areas.
    if lb_mesh is not None:
        points = [from_point3d(pt) for pt in lb_mesh.face_centroids]

    return points

def convert_to_grid( _positions, _name_ = '', _directions_ = [], mesh_ = None, base_geo_ = None):
    """
    Create a Sensor Grid object that can be used in a grid-based recipe.
    -

        Args:
            _name_: A name for this sensor grid.
            _positions: A list or a datatree of points with one point for the position
                of each sensor. Each branch of the datatree will be considered as a
                separate sensor grid.
            _directions_: A list or a datatree of vectors with one vector for the
                direction of each sensor. The input here MUST therefor align with
                the input _positions. If no value is provided (0, 0, 1) will be
                assigned for all the sensors.
            mesh_: An optional mesh that aligns with the sensors. This is useful for
                generating visualizations of the sensor grid beyond the sensor
                positions. Note that the number of sensors in the grid must match
                the number of faces or the number vertices within the mesh.
            base_geo_: An optional Brep for the geometry used to make the grid. There are
                no restrictions on how this brep relates to the sensors and it is
                provided only to assist with the display of the grid when the number
                of sensors or the mesh is too large to be practically visualized.

        Returns:
            grid: An SensorGrid object that can be used in a grid-based recipe.
    """
    # set the default name and process the points to tuples
    pts = [(pt.X, pt.Y, pt.Z) for pt in _positions]

    # create the sensor grid object
    id  = _name_
    if len(_directions_) == 0:
        grid = SensorGrid.from_planar_positions(id, pts, (0, 0, 1))
    else:
        vecs = [(vec.X, vec.Y, vec.Z) for vec in _directions_]
        grid = SensorGrid.from_position_and_direction(id, pts, vecs)
        
    if mesh_ is not None:
        grid.mesh = to_mesh3d(mesh_)
    if base_geo_ is not None:
        grid.base_geometry = to_face3d(base_geo_)

    return grid

In [36]:
#FONCTION ANNUAL IRRADIANCE

def annual_irradiance(_model, _wea, _timestep_ = 1, visible_ = False, north_ = 0, grid_filter_ = None, radiance_par_ = "-ab 2 -ad 5000 -lw 2e-05", run_settings_ = None):
    """
    Run an annual irradiance study for a Honeybee model to compute hourly solar
    irradiance for each sensor in a model's sensor grids.
    _
    The fundamental calculation of this recipe is the same as that of "HB Annual
    Daylight" in that an enhaced 2-phase method is used to accurately account for
    direct sun at each simulation step. However, this recipe computes broadband
    solar irradiance in W/m2 instead of visible illuminance in lux.
    _
    Consequently, the average irradiance and cumulative radiation values produced from
    this recipe are more accurate than those produced by the "HB Cumulative Radiation"
    recipe. Furthermore, because the hourly irriadiance values are accurate, this
    recipe can be used to evaluate `peak_irradiance` and determine the worst-case
    solar loads over clear sky Weas that represent cooling design days.

    -
        Args:
            _model: A Honeybee Model for which Annual Irradiance will be simulated.
                Note that this model must have grids assigned to it.
            _wea: A Wea object produced from the Wea components that are under the Light
                Sources tab. This can also be the path to a .wea or a .epw file.
            _timestep_: An integer for the timestep of the inpput _wea. This value is used
                to compute average irradiance and cumulative radiation. (Default: 1)
            visible_: Boolean to indicate the type of irradiance output, which can be solar (False)
                or visible (True). Note that the output values will still be
                irradiance (W/m2) when "visible" is selected but these irradiance
                values will be just for the visible portion of the electromagnetic
                spectrum. The visible irradiance values can be converted into
                illuminance by multiplying them by the Radiance luminous efficacy
                factor of 179. (Default: False).
            north_: A number between -360 and 360 for the counterclockwise difference
                between the North and the positive Y-axis in degrees. This can
                also be Vector for the direction to North. (Default: 0).
            grid_filter_: Text for a grid identifer or a pattern to filter the sensor grids of
                the model that are simulated. For instance, first_floor_* will simulate
                only the sensor grids that have an identifier that starts with
                first_floor_. By default, all grids in the model will be simulated.
            radiance_par_: Text for the radiance parameters to be used for ray
                tracing. (Default: -ab 2 -ad 5000 -lw 2e-05).
            run_settings_: Settings from the "HB Recipe Settings" component that specify
                how the recipe should be run. This can also be a text string of
                recipe settings.
            _run: Set to True to run the recipe and get results. This input can also be
                the integer "2" to run the recipe silently.

        Returns:
            results: Raw result files (.ill) that contain matrices of irradiance in W/m2
                for each time step of the wea.
    """
    # Créer la recette et définir les arguments d'entrée
    recipe = Recipe('annual-irradiance')
    recipe.input_value_by_name('model', _model)
    recipe.input_value_by_name('wea', _wea)
    recipe.input_value_by_name('timestep', _timestep_)
    recipe.input_value_by_name('output-type', visible_)
    recipe.input_value_by_name('north', north_)
    recipe.input_value_by_name('grid-filter', grid_filter_)
    recipe.input_value_by_name('radiance-parameters', radiance_par_)

    if run_settings_ is not None :
        project_folder = recipe.run(run_settings_, radiance_check=True, silent=False)

    recipe.default_project_folder = os.path.join('C:','Users','maceo.valente','Documents','Automatisation','TEST_MVA','TSAgriPV')
    
    # load the results
    results = recipe_result(recipe.output_value_by_name('results', project_folder))

    return results


Paramétrage/Settings :

In [37]:
# Geometrical settings  ====> ACTUELLEMENT CONFIGURE POUR PIOLENC
######
GRID_SIZE = 50 # Taille du champ de panneau pour la simulation
ENTRAXE = 2.25 # Espacement entre lignes de panneaux (= inter-rang classique, distance entre 2 supports de panneaux)
HAUTEUR = 6 # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
RAMPANT = 1.134 # Largeur du panneau
NB_RANGS = 10 # Nombre de panneaux par ligne
NB_PVP_RANGS = int(GRID_SIZE / ENTRAXE) # Nombre de lignes
LONGUEUR_PVP = GRID_SIZE / (NB_RANGS) # Longueur des panneaux dans une même ligne
# Semitransp panels :
TYPE_PANEL = True #False = classic panel, True = semitransparent panel
LARGEUR_BANDE = 0.18
LARGEUR_AVIDE = (RAMPANT-4*LARGEUR_BANDE)/3

column_spacing = ENTRAXE # Espacement entre lignes de panneaux (= inter-rang)
row_spacing = GRID_SIZE / NB_RANGS # Espacement entre panneaux d'une même ligne (même support)

ANGLE_ORIENTATION = 10  # Rotation globale autour de l'axe Z (Orientation parcelle)

# Settings, year and hours
######
# HEURE_DEB = 10
# HEURE_FIN = 14
start_date = pd.Timestamp('2023-01-01')
# end_date = datetime(year=2024, month=1, day=1, hour=HEURE_DEB)  # Fin de l'année (début de l'année suivante)
# NB_HEURES_ANNEES = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days

# Settings angles
#####
start_angle = -60
end_angle = 60
step_angle = 5
angles = np.arange(start_angle, end_angle + 1, step_angle)
# angles = [-60,0,60]
print(list(angles))

# Settings radiance
#####
SKYMAT_FOLDER = "cielMatrices/"
HAUTEUR_MESURE = 1 # Hauteur de mesure dans IncidentRadiation
FINESSE = 0.5 # Finesse de la fonction création SensorGrid (Diviser 10 par FINESSE pour avoir le nombre de points)
_epw_file_path = "FRA_PR_Orange.Caritat.AB.075790_TMYx.epw" #Filepath to meteo file (EPW format)
timestep_wea = 1 # (hour)
path_resultsHB_AI = os.path.join('Annual_Irr_Results')
settings_HB_AI = RecipeSettings(path_resultsHB_AI)
settings_HB_AI_CT = RecipeSettings(path_resultsHB_AI+'_CT')

# Settings containers results
hoys = np.arange(0, 8760)
timestamps = [start_date + pd.Timedelta(hours=int(hour)) for hour in hoys]
if 'tab_1' in locals() :
    del(tab_1)
tab_1 = pd.DataFrame()
tab_1['Month-Day-Hour'] = [ts.strftime('%m-%d-%H') for ts in timestamps]

output_path = "bdd_irr/step60_semitransp_HBmethod.xlsx"


[-60, -55, -50, -45, -40, -35, -30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]


In [38]:
# #Generate all Rhino .3dm files corresponding to all the different tilt panels can have
# #Optionnal 

# panneaux_TEMOIN = rg.Brep()
# panneaux_TEMOIN.Append(create_surface(40, 40, 0.1, 0.1).ToBrep())
# sol_temoin = create_sensor_grid_2(ANGLE_ORIENTATION,40,40,culture=True)
# measures_grid = create_sensor_grid_2(ANGLE_ORIENTATION, None, None, culture=True, 
#                                     entraxe=ENTRAXE, rampant=RAMPANT, nb_lignes=NB_PVP_RANGS, grid_size=GRID_SIZE)
# for i in angles :
#     #VISU
#     file_path = "panneaux_PIOLENC/semitransp" + str(i) + ".3dm" 
#     p = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, i,
#                                         TYPE_PANEL, LARGEUR_BANDE, LARGEUR_AVIDE) #True/False : semitransparent panel or classic one
#     export_brep_to_file(p, file_path)
#     export_brep_to_file(measures_grid[0], file_path, False)
#     export_brep_to_file(panneaux_TEMOIN, file_path, False)
#     export_brep_to_file(sol_temoin[0], file_path, False)

Import EPW

In [39]:
# Here, the meteo file (epw format) is imported and wea is created from the epw file.
epw_data = epw.EPW(_epw_file_path)

wea = Wea.from_epw_file(_epw_file_path, timestep_wea)


# location_ = epw_data.location #######
# direct_normal_rad_ = epw_data.direct_normal_radiation #########
# diffuse_horizontal_rad_ = epw_data.diffuse_horizontal_radiation #########

Boucle 1 (= main)

In [40]:
### Running the simulation with control area : ------------------------------------------
#Creating classic geometry
panels_CT = rg.Brep()
panels_CT.Append(create_surface(40, 40, 0.1, 0.1).ToBrep())
ground_CT = create_sensor_grid_2(ANGLE_ORIENTATION,40,40,culture=True)

#Convert to Model control area
panels_CT_faces = brep_to_face([panels_CT])
panel_CT_model = convert_to_model(panels_CT_faces)
ground_CT_grid = brep_to_pts_mesh(ground_CT[0], _grid_size = FINESSE) #BREP TO POINTS
ground_CT_grid_sensor = convert_to_grid(ground_CT_grid,'ID_CT')
panel_CT_model_gridded = combine_grid_model(panel_CT_model, [ground_CT_grid_sensor]) #MODEL COMBINED WITH GRID
#Run the annual irradiance function 
results = annual_irradiance(_model=panel_CT_model_gridded, _wea= wea, run_settings_= settings_HB_AI_CT) 
### End of the CT simulation ------------------------------------------------------------

# The measurement grid does not change depending on the inclination of the panels, it can only be created once. culture = True : arboriculture
ground = create_sensor_grid_2(ANGLE_ORIENTATION, None, None, culture=True, 
                                    entraxe=ENTRAXE, rampant=RAMPANT, nb_lignes=NB_PVP_RANGS, grid_size=GRID_SIZE)
ground_grid = brep_to_pts_mesh(ground[0], _grid_size = FINESSE) #BREP TO POINTS
ground_grid_sensor = convert_to_grid(ground_grid,'ID_1')
 
#Boucle 1, Varying parameter : = tilt angle of PVPs
for index, angle in enumerate(angles):

    #Create panel geometry
    final_rotated_brep = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, angle,
                                           TYPE_PANEL, LARGEUR_BANDE, LARGEUR_AVIDE)
    
    #Convert to Model
    panel_faces = brep_to_face([final_rotated_brep])
    panel_model = convert_to_model(panel_faces)
    panel_model_gridded = combine_grid_model(panel_model, [ground_grid_sensor]) #MODEL COMBINED WITH GRID

    #Run annual irradiance
    results = annual_irradiance(_model=panel_model_gridded, _wea= wea, run_settings_= settings_HB_AI)

    #Export data to previously created container----------------
    # Step 1: Read the sun-up hours 
    sun_up_hours = pd.read_csv('Annual_Irr_Results/annual_irradiance/results/total/sun-up-hours.txt', header=None).squeeze()
    sun_up_hours = sun_up_hours - 0.5

    # Step 2: Read the .ill file and calculate the mean values for each column
    ill_data = pd.read_csv('Annual_Irr_Results/annual_irradiance/results/total/ID_1.ill', delim_whitespace=True, header=None)
    ill_data_mean = ill_data.mean()

    angle_txt = str(angle)
    tab_1[angle_txt] = 0
    for hour in hoys:
        if hour in sun_up_hours.values:
            index = sun_up_hours[sun_up_hours == hour].index[0]
            tab_1.at[hour,angle_txt] = ill_data_mean[index]
    #-----------------------------------------------------------
    #Supress temp datafiles
    #Erase HB_AI folder
    if os.path.exists(path_resultsHB_AI):
        # Supprimez le dossier et tout son contenu
        shutil.rmtree(path_resultsHB_AI)
        print(f"Le dossier '{path_resultsHB_AI}' a été supprimé avec succès.")
    else:
        print(f"Un des deux dossiers n'existe pas.")

#Add the control light sim results to the table
# Step 1: Read the sun-up hours 
sun_up_hours = pd.read_csv('Annual_Irr_Results_CT/annual_irradiance/results/total/sun-up-hours.txt', header=None).squeeze()
sun_up_hours = sun_up_hours - 0.5

# Step 2: Read the .ill file and calculate the mean values for each column
ill_data = pd.read_csv('Annual_Irr_Results_CT/annual_irradiance/results/total/ID_CT.ill', delim_whitespace=True, header=None)
ill_data_mean = ill_data.mean()

CT_txt = 'temoin'
tab_1[CT_txt] = 0
for hour in hoys:
    if hour in sun_up_hours.values:
        index = sun_up_hours[sun_up_hours == hour].index[0]
        tab_1.at[hour,CT_txt] = ill_data_mean[index]

#Erase HB_AI_CT folder
if os.path.exists(path_resultsHB_AI + '_CT'):
    # Supprimez le dossier et tout son contenu
    shutil.rmtree(path_resultsHB_AI + '_CT')
    print(f"Le dossier '{path_resultsHB_AI + '_CT'}' a été supprimé avec succès.")
else:
    print(f"Le dossier '{path_resultsHB_AI + '_CT'}' n'existe pas.")

#Export to excel file the result :
tab_1.to_excel(output_path, index=False, header=True)

Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été supprimé avec succès.
Le dossier 'Annual_Irr_Results' a été su

In [10]:
# #VISU
# file_path = "output_apres_boucle.3dm"
# export_brep_to_file(final_rotated_brep, file_path)
# export_brep_to_file(measures_grid[0], file_path, False)
# export_brep_to_file(panneaux_TEMOIN, file_path, False)
# export_brep_to_file(sol_temoin[0], file_path, False)

In [21]:
tab_1[0:25]

,Month-Day-Hour,-60,-55,-50,-45,-40,-35,-30,-25,-20,...,15,20,25,30,35,40,45,50,55,60
0,01-01-00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01-01-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01-01-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01-01-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,01-01-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,01-01-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,01-01-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,01-01-08,12,12,12,11,11,11,11,10,10,...,9,9,9,9,10,10,10,10,11,11
9,01-01-09,28,31,32,34,33,33,33,32,31,...,24,24,22,22,23,21,21,22,22,22
